# Gaussian Naive Bayes
A reimplementation of the `sklearn` Gaussian Naive Bayes classifier. 

1. Provide a python class MyGaussianNB that implements Gaussian Naive Bayes. 
The API specification for sklearn classifiers is here: https://scikit-learn.org/stable/developers/develop.html 
You should implement the ‘fit’ and ‘predict’ methods, there is no need to implement ‘predict_proba’. 


In [11]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
from sklearn.metrics import accuracy_score

## My GaussianNB
Reimplementation of a Gaussian Naive Bayes.

In [12]:
class MyGaussianNB(BaseEstimator, ClassifierMixin):          
    def fit(self, Xt, yt):
        self.var_smoothing = 1e-9   # zero variance will cause division by zero errors.
        self.Xt = Xt
        self.yt = yt
        self.n_feat = Xt.shape[1]
        self.mus = {}
        self.sig_sqs = {}
        self.priors = {}
        
        c_dict = Counter(self.yt)
        
        for c in c_dict.keys():
            self.mus[c] = np.zeros(self.n_feat) # where the means will be stored
            self.sig_sqs[c] = np.zeros(self.n_feat) # where the variances will be stored
            self.priors[c] = c_dict[c]/Xt.shape[0]
            
            mask = self.yt == c
            X_tr_c = self.Xt[mask, :] # the rows for this class label
            
            for f in range(self.n_feat):
                self.mus[c][f] = np.mean(X_tr_c[:,f])
                self.sig_sqs[c][f] = np.var(X_tr_c[:,f] + self.var_smoothing)  #var              
        #print(self.mus)
        #print(self.sig_sqs)
        
        return self
    
    # The predictions are the most common class in the training set.
    def predict(self, Xtes):
        #print("Predicting MGNB")
        self.Xtes = Xtes
         
        res_list = []
        for sample in Xtes:
            res_list.append(self.predict_single(sample))
            
        return np.array(res_list)
    
    def predict_single(self, x_single):
        probs = {}
        for c in self.priors.keys():   # for each of the class labels
            probs[c] = self.priors[c]
            for i, f in enumerate(x_single):
                t1 = 1/math.sqrt(2*math.pi*self.sig_sqs[c][i])
                num = (f - self.mus[c][i])**2
                den = 2*self.sig_sqs[c][i]
                pxi_y = t1 * math.exp(-num/den)
                probs[c] = probs[c] * pxi_y
                #print(t1, num, den, pxi_y)
                #print(probs)
            #print(c, self.priors[c])
        return max(probs, key=probs.get) # Return the key with the largest value
    

## Testing
2. Test the performance of your implementation against the `GaussianNB` implementation in `scikit-learn`. You should use a range of datasets for this testing.   
Four datasets are used for testing; testing on a hold out set:
 - **penguins**: check that mean and variance estimates are the same, check that predictions are the same. 
 - **diabetes**: check that predictions are the same.
 - **glassV2**: test that predictions are the same. 
 - **bike_sharing**: test that predictions are the same. 

The main component in the testing is to check the `fidelity` of the testing against the Gaussian Naive Bayes implementation in `scikit-learn`.    
The `fidelity_tests` function compares predictions across multiple runs (different holdout tests). It uses `accuracy_score` to do the comparison.

In [13]:
def fidelity_tests (X,y, nreps = 10):
    for rs in range(1, nreps + 1):
        X_tr_raw, X_ts_raw, y_train, y_test = train_test_split(X_raw, y, 
                                                               random_state=rs, 
                                                               test_size=1/2)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_tr_raw)
        X_test = scaler.transform(X_ts_raw)
        gnb = GaussianNB()
        mgnb = MyGaussianNB()
        mgnb.fit(X_train,y_train)
        gnb.fit(X_train,y_train)
        ascore = accuracy_score(gnb.predict(X_test),mgnb.predict(X_test)) 
        gacc = accuracy_score(gnb.predict(X_test),y_test)
        macc = accuracy_score(mgnb.predict(X_test),y_test)
        print ("Run: %d Score: %.2f SK acc: %.2f My acc: %.2f" % (rs, ascore, gacc, macc))

### Penguins

In [17]:
penguins = pd.read_csv('penguins.csv', index_col = 0)
print(penguins.shape)
penguins.head()

(333, 5)


,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,39.1,18.7,181,3750
1,Adelie,39.5,17.4,186,3800
2,Adelie,40.3,18.0,195,3250
4,Adelie,36.7,19.3,193,3450
5,Adelie,39.3,20.6,190,3650


In [18]:
y = penguins.pop('species').values
X_raw = penguins.values
X_tr_raw, X_ts_raw, y_train, y_test = train_test_split(X_raw, y, random_state=2, test_size=1/2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_tr_raw)
X_test = scaler.transform(X_ts_raw)
max_k = X_train.shape[1]
X_train.shape, X_test.shape

((166, 4), (167, 4))

**Model Parameters**  
The means are the same:

In [19]:
gnb = GaussianNB()
gnb.fit(X_train,y_train)
mgnb = MyGaussianNB()
mgnb.fit(X_train,y_train)

MyGaussianNB()

In [20]:
gnb.var_

array([[0.23452725, 0.4081271 , 0.20267081, 0.35176248],
       [0.31689883, 0.31237582, 0.20405537, 0.24846588],
       [0.28810086, 0.25004371, 0.27702916, 0.40919055]])

In [21]:
mgnb.sig_sqs

{'Chinstrap': array([0.31689883, 0.31237582, 0.20405537, 0.24846588]),
 'Gentoo': array([0.28810086, 0.25004371, 0.27702916, 0.40919054]),
 'Adelie': array([0.23452725, 0.4081271 , 0.20267081, 0.35176248])}

In [22]:
gnb.theta_

array([[-0.92625328,  0.52195557, -0.77406098, -0.59102787],
       [ 0.91764385,  0.70847529, -0.24527437, -0.53946387],
       [ 0.69991712, -1.13851399,  1.19942791,  1.12964089]])

In [23]:
mgnb.mus

{'Chinstrap': array([ 0.91764385,  0.70847529, -0.24527437, -0.53946387]),
 'Gentoo': array([ 0.69991712, -1.13851399,  1.19942791,  1.12964089]),
 'Adelie': array([-0.92625328,  0.52195557, -0.77406098, -0.59102787])}

Accuracy scores are the same:

In [24]:
gnb.score(X_test, y_test)

0.9640718562874252

In [25]:
mgnb.score(X_test, y_test)

0.9640718562874252

### Fidelity tests

Look at the lables of the predictions of the first 10 test samples:

In [26]:
mgnb.predict(X_test[:10])

array(['Chinstrap', 'Gentoo', 'Adelie', 'Gentoo', 'Adelie', 'Adelie',
       'Gentoo', 'Adelie', 'Adelie', 'Adelie'], dtype='<U9')

In [27]:
gnb.predict(X_test[:10])

array(['Chinstrap', 'Gentoo', 'Adelie', 'Gentoo', 'Adelie', 'Adelie',
       'Gentoo', 'Adelie', 'Adelie', 'Adelie'], dtype='<U9')

Run multiple tests

In [28]:
fidelity_tests(X_raw, y)

Run: 1 Score: 1.00 SK acc: 0.94 My acc: 0.94
Run: 2 Score: 1.00 SK acc: 0.96 My acc: 0.96
Run: 3 Score: 1.00 SK acc: 0.95 My acc: 0.95
Run: 4 Score: 1.00 SK acc: 0.95 My acc: 0.95
Run: 5 Score: 1.00 SK acc: 0.96 My acc: 0.96
Run: 6 Score: 1.00 SK acc: 0.96 My acc: 0.96
Run: 7 Score: 1.00 SK acc: 0.98 My acc: 0.98
Run: 8 Score: 1.00 SK acc: 0.97 My acc: 0.97
Run: 9 Score: 1.00 SK acc: 0.96 My acc: 0.96
Run: 10 Score: 1.00 SK acc: 0.99 My acc: 0.99


Finally, we use `accuracy_score` to compare all predictions on the test set.   
The score of 1.0 indicates perfect agreement.

### Diabetes dataset  
Test that the predictions are the same on a holdout set. 

In [29]:
diabetes = pd.read_csv('diabetes.csv') #, index_col = 0)
print(diabetes.shape)
diabetes.head()

(768, 9)


,preg,plas,pres,skin,insu,mass,pedi,age,neg_pos
0,6,148,72,35,0,33.6,0.627,50,tested_positive
1,1,85,66,29,0,26.6,0.351,31,tested_negative
2,8,183,64,0,0,23.3,0.672,32,tested_positive
3,1,89,66,23,94,28.1,0.167,21,tested_negative
4,0,137,40,35,168,43.1,2.288,33,tested_positive


In [30]:
y = diabetes.pop('neg_pos').values
X_raw = diabetes.values

In [31]:
fidelity_tests(X_raw, y, nreps = 5)

Run: 1 Score: 1.00 SK acc: 0.74 My acc: 0.74
Run: 2 Score: 1.00 SK acc: 0.74 My acc: 0.74
Run: 3 Score: 1.00 SK acc: 0.74 My acc: 0.74
Run: 4 Score: 1.00 SK acc: 0.77 My acc: 0.77
Run: 5 Score: 1.00 SK acc: 0.73 My acc: 0.73


### Glass Dataset
Test that the predictions are the same on a holdout set. 

In [33]:
glass = pd.read_csv('glassV2.csv') #, index_col = 0)
print(glass.shape)
glass.head()

(205, 10)


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [34]:
glass_orig = pd.read_csv('glassV2.csv')

In [35]:
glass_orig['Type'].value_counts()

2    76
1    70
7    29
3    17
5    13
Name: Type, dtype: int64

In [36]:
glass = glass_orig[glass_orig['Type'].isin([1,2])]

In [37]:
y = glass.pop('Type')
X_raw = glass.values

In [38]:
fidelity_tests(X_raw, y, nreps = 10)

Run: 1 Score: 1.00 SK acc: 0.60 My acc: 0.60
Run: 2 Score: 1.00 SK acc: 0.55 My acc: 0.55
Run: 3 Score: 1.00 SK acc: 0.62 My acc: 0.62
Run: 4 Score: 0.99 SK acc: 0.64 My acc: 0.66
Run: 5 Score: 1.00 SK acc: 0.62 My acc: 0.62
Run: 6 Score: 1.00 SK acc: 0.59 My acc: 0.59
Run: 7 Score: 1.00 SK acc: 0.64 My acc: 0.64
Run: 8 Score: 1.00 SK acc: 0.55 My acc: 0.55
Run: 9 Score: 1.00 SK acc: 0.63 My acc: 0.63
Run: 10 Score: 1.00 SK acc: 0.60 My acc: 0.60


One miss-match here. 

## Bike Sharing

In [39]:
bikes_df = pd.read_csv('bike_sharing.csv')
bikes_df.head()
bikes_df['usage'] = 'Low'
bikes_df.loc[bikes_df['count'] > 4500, 'usage'] = 'High'

In [40]:
bikes_df['usage'].value_counts()

High    372
Low     359
Name: usage, dtype: int64

In [41]:
y = bikes_df.pop('usage').values
bikes_df.pop('casual').values
bikes_df.pop('registered').values
bikes_df.pop('instant').values
bikes_df.pop('dteday').values
bikes_df.pop('count').values
X_raw = bikes_df.values
X_raw.shape, y.shape

((731, 11), (731,))

In [42]:
fidelity_tests(X_raw, y, nreps = 10)

Run: 1 Score: 1.00 SK acc: 0.83 My acc: 0.83
Run: 2 Score: 1.00 SK acc: 0.80 My acc: 0.80
Run: 3 Score: 1.00 SK acc: 0.78 My acc: 0.78
Run: 4 Score: 1.00 SK acc: 0.79 My acc: 0.79
Run: 5 Score: 1.00 SK acc: 0.80 My acc: 0.80
Run: 6 Score: 1.00 SK acc: 0.81 My acc: 0.81
Run: 7 Score: 1.00 SK acc: 0.78 My acc: 0.78
Run: 8 Score: 1.00 SK acc: 0.84 My acc: 0.84
Run: 9 Score: 1.00 SK acc: 0.84 My acc: 0.84
Run: 10 Score: 1.00 SK acc: 0.83 My acc: 0.83
